In [45]:
import pdfplumber 
import pandas as pd
import time
from datetime import datetime, timedelta
from string import digits

import docx
from docx.shared import RGBColor
from docx.shared import Cm, Pt  #加入可調整的 word 單位
from docx.oxml.ns import qn
from docx.enum.text import WD_ALIGN_PARAGRAPH #處理字串的置中
from docx.enum.section import WD_ORIENT 
price=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
vendor=[]
vendna=set()
delta = ["減為","維持","未回覆"]
pdf = pdfplumber.open("C:\\Users\\JerryLiu\\Desktop\\0921\\0930\\0923test.pdf") 

#新增word
doc = docx.Document()
doc.add_heading('議價流程表')
section = doc.sections[0] # 節
section.orientation = WD_ORIENT.LANDSCAPE # 調整橫式紙張
new_width, new_height = Cm(29.7), Cm(21)
section.page_width = new_width
section.page_height= new_height
for paragraph in doc.paragraphs:
    if paragraph.style.name.startswith('Heading'):
        paragraph.paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER # LEFT, CENTER, RIGHT
        for run in paragraph.runs:
            run.font.color.rgb = RGBColor(0, 0, 0)
            run.font.name = '標楷體'
            run._element.rPr.rFonts.set(qn('w:eastAsia'), u'標楷體')
            run.font.size = Pt(18)

doc.add_paragraph("工程編號：XXX")
doc.add_paragraph("工程名稱：XXXX工程")
doc.add_paragraph(" ")
doc.add_paragraph("2020/05/26  接件、上網詢價詢止日2020/06/28。")
doc.add_paragraph("2020/06/29  開價格標，XX家廠商「XX」、「XX」參與報價。")
doc.add_paragraph("2020/07/08  1000萬元(含)以上重大案件工程檢討會議，檢討內容如下：開標計X家報價，報價最低廠商「XX」，\
較預算高+XXX%，經會議結論擬以總價XXX元以下，向廠商「XX」議價。")
doc.add_paragraph(" ")
                  
####judge_logic(rtable[ii][2],rtable[ii][3],rtable[ii][4],rtable[ii][5],length)
def judge_logic(quote_price,final_nego_price,target_price,recive_price,length,num):
    delta = ["減為","維持","未回覆"]
    tempdelta = ""
    if int(final_nego_price.replace(",","")) == 0: ###第一次議後金額為零的結果
        if int(recive_price.replace(",","")) == int(target_price.replace(",","")) and len(length) ==8:
            tempdelta = delta[2]   ###議後未回覆的結果
        elif int(recive_price.replace(",","")) < int(quote_price.replace(",","")) : ##delta = [減為,,]         
            #price[num][2]=recive_price
            tempdelta = "{}{}元".format(delta[0],recive_price)
        elif int(recive_price.replace(",","")) == int(quote_price.replace(",","")) :##delta = [,維持,] 
            tempdelta = "{}{}元".format(delta[1],recive_price)                    
    ###議後未回覆的結果
    elif int(recive_price.replace(",","")) == int(target_price.replace(",","")) and len(length) ==8:
        tempdelta = delta[2] 
    ##進入判斷減為、維持的迴圈    
    elif int(final_nego_price.replace(",","")) != 0: 
        if int(recive_price.replace(",","")) < int(final_nego_price.replace(",","")) : ##delta = [減為,,]           
            #price[num][2]=recive_price
            tempdelta = "{}{}元".format(delta[0],recive_price)
        elif int(recive_price.replace(",","")) == int(final_nego_price.replace(",","")) :##delta = [,維持,]
            tempdelta = "{}{}元".format(delta[1],recive_price)
    return tempdelta

k=0
for i in range(len(pdf.pages)):
    rtable=[]         
    p0 = pdf.pages[i]
    table = p0.extract_table()
    for ele in table:
        a = list(filter(None,ele))###去空白欄位
        rtable.append(a)        
        newtable=[]###remove"\n換行符號"
        for ele in rtable:
            rrtable=[]            
            for elee in ele:
                rrtable.append(elee.replace("\n",""))                
            newtable.append(rrtable)
    ii=2
    j=0

    for i in range(int((len(rtable)-2)/2)):###用行數減初始值2再除以2做為議價欄位出現次數        
        ###時間格式轉換
        date = datetime.strptime(str(rtable[ii][6]), '%Y%m%d')
        timeArray = time.strptime(str(date), "%Y-%m-%d %H:%M:%S")
        rtable[ii][6]= time.strftime("%Y/%m/%d", timeArray)
        ###刪除廠商統編
        remove_digits = str.maketrans('', '', digits)
        newtable[ii][1] =newtable[ii][1].translate(remove_digits)       
        
        if price[j]==[]:
            price[j].append(newtable[ii][1])
            price[j].append(newtable[ii][2])
            price[j].append(newtable[ii][5])
            vendna.add(newtable[ii][1])
            
        for j in range(j+1):     
            if newtable[ii][1] == price[j][0]: 
                judge_logic(rtable[ii][2],rtable[ii][3],rtable[ii][4],rtable[ii][5],rtable[ii],j)
            if newtable[ii][1] != price[j][0]:
                if (newtable[ii][1] in vendna) == False:
                    k+=1               
                    price[k].append(newtable[ii][1])
                    price[k].append(newtable[ii][2])
                    price[k].append(newtable[ii][5])
                    vendna.add(newtable[ii][1])
                    if newtable[ii][1] == price[k][0]:
                        judge_logic(rtable[ii][2],rtable[ii][3],rtable[ii][4],rtable[ii][5],rtable[ii],k)
   
        para=("{} (網路議價『議別{}』出價{}元) 「{}」{}。".format(rtable[ii][6],rtable[ii][0],rtable[ii][4],newtable[ii][1],\
                                                    judge_logic(rtable[ii][2],rtable[ii][3],rtable[ii][4],rtable[ii][5],\
                                                                rtable[ii],k)))
        doc.add_paragraph(para)
        
#         print("{} (網路議價『議別{}』出價{}元) 「{}」{}。".format(rtable[ii][6],rtable[ii][0],rtable[ii][4],newtable[ii][1],\
#                                                     judge_logic(rtable[ii][2],rtable[ii][3],rtable[ii][4],rtable[ii][5],\
#                                                                 rtable[ii],k)))
        ii=ii + 2
doc.add_paragraph("2020/09/22 擬以XXX元交由「XX」、「XX」及「XX」共同承攬，並OA決包會簽工程部門確認。")
doc.add_paragraph("2020/09/25 工程部門決包會簽回覆：同意決包。")    
doc.add_paragraph("2020/09/25 工程決包單呈核。")    
doc.add_paragraph("                                  主管:                   經辦:")
for paragraph in doc.paragraphs:
    for run in paragraph.runs:
        run.font.color.rgb = RGBColor(0, 0, 0)
        run.font.name = '標楷體'
        run._element.rPr.rFonts.set(qn('w:eastAsia'), u'標楷體')
        run.font.size = Pt(16)
doc.save('議價流程表.docx')

In [130]:
# for ele in newtable:
#     print(ele)
# print(newtable)
vendor=["我","是","誰"]
# print(vendor[0])
# for ele in rtable:
#     print(ele)

In [133]:
def test(name,age,remark="good"):
    print(name)
    print("age:{}".format(age))
    print("remark is {}".format(remark))
    return "done"

test("jerryliu",18)

jerryliu
age:18
remark is good


'done'